In [1]:
from matplotlib import pyplot as plt
import seaborn as sns

import pandas as pd
import datetime
import swifter
import numpy as np

from tqdm.notebook import tqdm

In [2]:
path_to_data = '../data/'

In [3]:
columns_to_read = ['timestamp', 'application_id', 'client', 'session_id', 'event_type', 'event_category', 
                   'event_name', 'event_label', 'page_urlhost', 'net_connection_type', 'net_connection_tech', 
                   'device_screen_name', 'device_is_webview']

In [4]:
filename = 'part-00000'

In [5]:
sample_click_stream = pd.read_parquet(path_to_data + 'alfabattle2_abattle_clickstream/' + filename + '.parquet', columns=columns_to_read)

In [6]:
sample_click_stream.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12468595 entries, 0 to 12468594
Data columns (total 13 columns):
 #   Column               Non-Null Count     Dtype         
---  ------               --------------     -----         
 0   timestamp            12468595 non-null  datetime64[ns]
 1   application_id       12468595 non-null  object        
 2   client               12468595 non-null  object        
 3   session_id           12468595 non-null  object        
 4   event_type           12468595 non-null  object        
 5   event_category       6766054 non-null   object        
 6   event_name           6682190 non-null   object        
 7   event_label          1751796 non-null   object        
 8   page_urlhost         437725 non-null    object        
 9   net_connection_type  12030870 non-null  object        
 10  net_connection_tech  7000769 non-null   object        
 11  device_screen_name   9665011 non-null   object        
 12  device_is_webview    12243467 non-null  

In [7]:
sample_click_stream = sample_click_stream.sort_values(['client', 'session_id', 'timestamp'])

### Last application_id

In [8]:
last_app_id_df = sample_click_stream.groupby(['client', 'session_id'])['application_id'].last().reset_index()

In [9]:
last_app_id_df.head(1)

,client,session_id,application_id
0,0014a49ec89e3a43098375b107f8ff2e,13c9eab5215d015776acb80675f9d70d,mobile


In [10]:
last_app_id_df = last_app_id_df.rename(columns={'application_id': 'last_application_id'})

### n_unique_app_id

In [11]:
nunique_app_id_df = sample_click_stream.groupby(['client', 'session_id'])['application_id'].nunique().reset_index()

In [12]:
nunique_app_id_df = nunique_app_id_df.rename(columns={'application_id': 'nunique_application_id'})

In [13]:
result_df = last_app_id_df.merge(nunique_app_id_df, how='inner', left_on=['client', 'session_id'], right_on=['client', 'session_id'])

### Session duration

In [14]:
duration_df = sample_click_stream.groupby(['client', 'session_id'])['timestamp'].agg(['min', 'max']).diff(axis=1).reset_index()
duration_df['duration'] = duration_df['max'].swifter.apply(lambda x: x.total_seconds())
duration_df = duration_df.drop(['min', 'max'], axis=1)

In [15]:
result_df = result_df.merge(duration_df, how='inner', left_on=['client', 'session_id'], right_on=['client', 'session_id'])

### Steps

In [16]:
steps_df = sample_click_stream.groupby(['client', 'session_id']).size().reset_index()

In [17]:
steps_df = steps_df.rename(columns={0: 'steps'})

In [18]:
result_df = result_df.merge(steps_df, how='inner', left_on=['client', 'session_id'], right_on=['client', 'session_id'])

### Last event category

In [19]:
last_event_cat_df = sample_click_stream.groupby(['client', 'session_id'])['event_category'].last().reset_index()
last_event_cat_df = last_event_cat_df.rename(columns={'event_category': 'last_event_category'})

In [20]:
result_df = result_df.merge(last_event_cat_df, how='inner', left_on=['client', 'session_id'], right_on=['client', 'session_id'])

### nunique_event_category

In [21]:
nunique_event_cat_df = sample_click_stream.groupby(['client', 'session_id'])['event_category'].nunique().reset_index()
nunique_event_cat_df = nunique_event_cat_df.rename(columns={'event_category': 'nunique_event_category'})

In [22]:
result_df = result_df.merge(nunique_event_cat_df, how='inner', left_on=['client', 'session_id'], right_on=['client', 'session_id'])

### nunique_event_label

In [23]:
nunique_event_label_df = sample_click_stream.groupby(['client', 'session_id'])['event_label'].nunique().reset_index()
nunique_event_label_df = nunique_event_label_df.rename(columns={'event_label': 'nunique_event_label'})
result_df = result_df.merge(nunique_event_label_df, how='inner', left_on=['client', 'session_id'], right_on=['client', 'session_id'])

### Max timestamp

In [24]:
max_timestamp_df = sample_click_stream.groupby(['client', 'session_id'])['timestamp'].max().reset_index()
max_timestamp_df = max_timestamp_df.rename(columns={'timestamp': 'max_timestamp'})
result_df = result_df.merge(max_timestamp_df, how='inner', left_on=['client', 'session_id'], right_on=['client', 'session_id'])

In [25]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1004032 entries, 0 to 1004031
Data columns (total 10 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   client                  1004032 non-null  object        
 1   session_id              1004032 non-null  object        
 2   last_application_id     1004032 non-null  object        
 3   nunique_application_id  1004032 non-null  int64         
 4   duration                1004032 non-null  float64       
 5   steps                   1004032 non-null  int64         
 6   last_event_category     928044 non-null   object        
 7   nunique_event_category  1004032 non-null  int64         
 8   nunique_event_label     1004032 non-null  int64         
 9   max_timestamp           1004032 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4), object(4)
memory usage: 84.3+ MB


In [26]:
result_df['last_event_category'] = result_df['last_event_category'].fillna('missed')

### add labels

In [27]:
train_df = pd.read_csv(path_to_data + 'alfabattle2_abattle_train_target.csv', parse_dates=['timestamp'])

In [28]:
train_df.head(1)

,session_id,client_pin,timestamp,multi_class_target
0,0000029e72e5fcde6a9f29c3a3ed198f,7cf9221322a0e2fdefb1b998b8f2ab29,2020-06-15 14:01:12,main_screen


In [29]:
result_df = result_df.merge(train_df.drop(['timestamp'], axis=1), how='left', left_on=['client', 'session_id'], right_on=['client_pin', 'session_id'])

In [30]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1004032 entries, 0 to 1004031
Data columns (total 12 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   client                  1004032 non-null  object        
 1   session_id              1004032 non-null  object        
 2   last_application_id     1004032 non-null  object        
 3   nunique_application_id  1004032 non-null  int64         
 4   duration                1004032 non-null  float64       
 5   steps                   1004032 non-null  int64         
 6   last_event_category     1004032 non-null  object        
 7   nunique_event_category  1004032 non-null  int64         
 8   nunique_event_label     1004032 non-null  int64         
 9   max_timestamp           1004032 non-null  datetime64[ns]
 10  client_pin              526700 non-null   object        
 11  multi_class_target      526700 non-null   object        
dtypes: datetime64[

In [31]:
result_df = result_df.drop(['client_pin'], axis=1)

In [32]:
result_df.to_csv(path_to_data + 'sessions_features/' + filename + '.csv', index=False)